# Map proteins (from Schmidt 2015) to reactions in the model

In [1]:
import pandas as pd
import cobra

from io import BytesIO
import httpx

data = BytesIO()
url = "http://bigg.ucsd.edu/static/models/iML1515.json"
client = httpx.Client()
with client.stream("GET", url) as response:
    for chunk in response.iter_bytes():
        data.write(chunk)        
client.close()

data.seek(0)
model = cobra.io.load_json_model(data)

In [2]:
gene_data = []
for rxn in model.reactions:
    for gen in rxn.genes:
        if gen.name:
            gene_data.append((gen.name, gen.id, rxn.id))
gene_df = pd.DataFrame(gene_data, columns=["gene_name", "gene_bnumber", "bigg.reaction"])
gene_df.sample(5)

,gene_name,gene_bnumber,bigg.reaction
159,lpxB,b0182,LPADSS
2247,ompC,b2215,ASCBtex
2360,argB,b3959,ACGK
4310,phoE,b0241,CRNDtex
1533,phoE,b0241,ETHSO3tex


In [3]:
rxn_df = pd.DataFrame.from_dict({rxn.id: rxn.annotation for rxn in model.reactions}).transpose()
rxn_df.drop("bigg.reaction", axis=1, inplace=True)
rxn_df.index.name = "bigg.reaction"
rxn_df.sample(5)

,biocyc,ec-code,kegg.reaction,metanetx.reaction,rhea,sbo,seed.reaction,sabiork,reactome.reaction
bigg.reaction,,,,,,,,,
MAN6Ptex,NaN,NaN,NaN,[MNXR101385],NaN,SBO:0000185,"[rxn13644, rxn08880]",NaN,NaN
CLPNH161pp,NaN,NaN,NaN,[MNXR96778],NaN,SBO:0000176,NaN,NaN,NaN
EX_ala_B_e,NaN,NaN,NaN,[MNXR95682],NaN,SBO:0000627,[rxn08175],NaN,"[R-RNO-909763, R-HSA-909765]"
PYK2,NaN,[2.7.1.40],[R00659],[MNXR103372],NaN,SBO:0000176,[rxn00460],[53],NaN
EX_btn_e,[META:TRANS-RXN0-240],NaN,NaN,[MNXR96333],"[28459, 28461, 28460, 28458]",SBO:0000627,NaN,NaN,NaN


In [4]:
# load the protein data (in copies per cell)
CONDITIONS = ["acetate", "galactose", "glucose", "glycerol", "pyruvate", "succinate"]
protein_df = pd.read_csv("protein_concentration_schmidt_2015.csv", index_col=0)[CONDITIONS]
protein_df = gene_df.join(protein_df, on="gene_name").drop(["gene_name", "gene_bnumber"], axis=1).groupby("bigg.reaction").sum()
protein_df.to_csv("protein_concentration_schmidt_2015_by_bigg_reactions.csv")